In [25]:
# SCHEDULED GENERATION AND AVAILABILITY

import pandas as pd
import numpy as np

from entsoe import EntsoePandasClient
from entsoe.mappings import Area, NEIGHBOURS, lookup_area
from entsoe.parsers import parse_generation
from typing import Union, Optional, Dict, List, Literal

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [27]:
start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
end = pd.Timestamp('20230331', tz='Europe/Amsterdam')

countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

In [28]:
df_plants = pd.read_excel('./plants.xlsx', index_col=0)
df_gen = pd.read_excel('../cost_curves/data/generation.xlsx', index_col=0)

In [29]:
display(df_plants)

,name,eic,type,zone,installed_capacity,lat,lon
0,GDK-Mellach,14W-WGM-KW-----J,natural_gas,AT,838.0,46.911700,15.488300
1,Ottensheim-Wilhering,14W-BOW-KW-----9,hydro_run_of_river,AT,179.0,48.316500,14.151057
2,FHKW Linz Mitte,14WLSG-MITTE---U,natural_gas,AT,216.8,48.299219,14.322935
3,Kraftwerk Theiß,14W-KW-THE-EVN-7,natural_gas,AT,485.0,48.394861,15.710147
4,FHKW Linz Süd,14WLSG-1SUED---5,natural_gas,AT,158.0,48.267893,14.346898
...,...,...,...,...,...,...,...
636,Petrokemija Kutina,NaN,natural_gas,HR,35.0,45.471290,16.792320
637,Dubrovnik,NaN,hydro_reservoir,HR,216.0,42.603001,18.235001
638,Fuzine,NaN,hydro_pumped_storage,HR,5.7,45.304985,14.714837
639,He Rijeka,NaN,hydro_run_of_river,HR,36.8,45.334999,14.452000


In [30]:
df_outages = pd.DataFrame()

for country_code in [x for x in countries if x not in ['HR', 'SI']]:
    print(country_code)
    df_outages_temp = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
    df_outages_temp = df_outages_temp.tz_localize(None)
    df_outages_temp['end'] = df_outages_temp['end'].dt.tz_localize(None)
    df_outages_temp['start'] = df_outages_temp['start'].dt.tz_localize(None)
    df_outages = pd.concat([df_outages, df_outages_temp], ignore_index=True)

display(df_outages)

AT
BE
CZ
DE_LU
FR
HU
NL
PL


Connection Error, retrying in 0 seconds


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [32]:
for country_code in ['PL', 'RO', 'SK']:
    print(country_code)
    df_outages_temp = client.query_unavailability_of_generation_units(country_code, start=start, end=end)
    df_outages_temp = df_outages_temp.tz_localize(None)
    df_outages_temp['end'] = df_outages_temp['end'].dt.tz_localize(None)
    df_outages_temp['start'] = df_outages_temp['start'].dt.tz_localize(None)
    df_outages = pd.concat([df_outages, df_outages_temp], ignore_index=True)

PL
RO
SK


,avail_qty,biddingzone_domain,businesstype,curvetype,docstatus,end,mrid,nominal_power,plant_type,production_resource_id,production_resource_location,production_resource_name,pstn,qty_uom,resolution,revision,start
0,389,AT,Planned maintenance,A03,Cancelled,2022-09-28 00:00:00,AG_5D27uQcpgx6crlpQsKg,840.0,Fossil Gas,14W-PROD-SIM---P,intra_zonal,KW Simmering,1,MAW,PT15M,3,2022-09-16 00:00:00
1,0,AT,Planned maintenance,A03,Cancelled,2024-01-01 00:00:00,_LluwDqTiRSOQFu38pGrGw,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,2,2023-01-01 00:00:00
2,0,AT,Planned maintenance,A03,Cancelled,2023-01-01 00:00:00,be3Cxp6yr6iQNN07LZSgJA,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,2,2022-01-01 00:00:00
3,0,AT,Planned maintenance,A03,None,2024-01-01 00:00:00,RLThCIJuxMYLJYdEKwL7eQ,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,1,2023-01-01 00:00:00
4,0,AT,Planned maintenance,A03,None,2023-01-01 00:00:00,KqPp9Q65jWB93v_xwyKxEA,140.0,Fossil Gas,14W-PROD-LAU---8,intra_zonal,KW Leopoldau,1,MAW,PT15M,1,2022-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16385,0,SK,Planned maintenance,A03,None,2023-02-24 12:00:00,5XX-YkWTHcEd7oTClJTm4w,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,39,2023-02-24 11:00:00
16386,0,SK,Planned maintenance,A03,None,2023-02-24 10:00:00,6zbRH9yTF8UGN9SVt16lEA,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,39,2023-02-24 09:00:00
16387,0,SK,Planned maintenance,A03,None,2023-01-24 14:00:00,0pDus7KsJotf7w5SgjOL8A,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,68,2023-01-24 07:00:00
16388,0,SK,Planned maintenance,A03,None,2023-03-01 17:00:00,9v_NDHwUlLbKn7RYoiAqSQ,122.4,Hydro Pumped Storage,24WV--HCV------O,Čierny Váh,Čierny Váh,1,MAW,PT60M,39,2023-03-01 08:00:00


In [36]:
df_generations = {}
end2 = pd.Timestamp('20220902', tz='Europe/Amsterdam')
for country_code in [x for x in countries if x not in []]:
    print(country_code)
    df_generation_country = client.query_generation_per_plant(country_code, start=start, end=end2)
    df_generation_country = df_generation_country.tz_localize(None)
    df_generations[country_code] = df_generation_country

display(df_generations['NL'])

AT
BE
CZ
DE_LU


InvalidBusinessParameterError: 

In [34]:
availability_matrix = np.zeros((len(df_gen), len(df_plants)))

for i, (p_index, plant) in enumerate(df_plants.iterrows()):
    print(i)
    available_capacity = plant['installed_capacity']
    df_plant_outages = df_outages[(df_outages.docstatus != 'Cancelled') & (df_outages.production_resource_id == plant['eic'])]
    has_outage = len(df_plant_outages) > 0
    
    for j, (t_index, time_row) in enumerate(df_gen.iterrows()):
        if has_outage:
            df_plant_outage = df_plant_outages[(df_plant_outages.start <= t_index) & (df_plant_outages.end >= t_index)]
            if len(df_plant_outage) > 0:
                available_capacity = df_plant_outage.iloc[0]['avail_qty']

        availability_matrix[j][i] = available_capacity

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
display(availability_matrix)

with open('availability_matrix_temp.npy', 'wb') as f:
    np.save(f, availability_matrix)

array([[838. , 179. , 216.8, ...,   5.7,  36.8, 276. ],
       [838. , 179. , 216.8, ...,   5.7,  36.8, 276. ],
       [838. , 179. , 216.8, ...,   5.7,  36.8, 276. ],
       ...,
       [  0. ,  80. , 216.8, ...,   5.7,  36.8, 276. ],
       [  0. ,  80. , 216.8, ...,   5.7,  36.8, 276. ],
       [  0. ,  80. , 216.8, ...,   5.7,  36.8, 276. ]])

In [ ]:
generation_matrix = np.zeros((len(df_gen), len(df_plants)))

for i, (p_index, plant) in enumerate(df_plants.iterrows()):
    print(i)
    for j, (t_index, time_row) in enumerate(df_gen.iterrows()):
        generation_matrix[j][i] = df_generations[plant['zone']].loc[t_index, plant['name']]